In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import pandas as pd


In [2]:
from bs4 import BeautifulSoup

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
import pymorphy2 # Морфологический анализатор.

In [4]:
# отключаем warnings
import warnings

warnings.filterwarnings('ignore')

In [5]:
# cleantext = BeautifulSoup(raw_html, "lxml").text

In [6]:
df_train_raw = pd.read_csv('data/train.csv', sep='\t')

In [7]:
df_train_raw.head()

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [8]:
df_train_raw.shape

(200000, 4)

In [9]:
df_test = pd.read_csv('data/test.csv', sep='\t')

In [10]:
df_test.head()

,id,name,description
0,200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...
1,200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...
2,200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...
4,200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...


In [37]:
df_test.shape

(170179, 3)

In [11]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
    

In [12]:
def prepare_data(dataframe):
    df = dataframe.copy()
    # clearhtml
    df['description'] = df['description'].apply(lambda x : cleanhtml(x) )
    # replace &quot to "
    df['description'] = df['description'].str.replace('&quot;','"')
    df['description'] = df['description'].str.replace('\u200b','')
    # uni name and description
    df['text'] = df['name'] + ' ' + df['description'] 
    # drop
    df = df.drop(columns=['name', 'description', 'id'], axis=1)
    return df
    

In [13]:
def f_tokenizer(s):
    morph = pymorphy2.MorphAnalyzer()
    if type(s) == str:
        t = s.split(' ')
    else:
        t = s
    f = []
    for j in t:
        m = morph.parse(j.replace('.',''))
        if len(m) != 0:
            wrd = m[0]
            if wrd.tag.POS not in ('NUMR','PREP','CONJ','PRCL','INTJ'):
                f.append(wrd.normal_form)
    return f

In [14]:
N = df_train_raw.shape[0]

In [15]:
%%time 
df_train = prepare_data(df_train_raw[:N])

CPU times: user 11.2 s, sys: 1.09 s, total: 12.3 s
Wall time: 12.4 s


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
target    200000 non-null int64
text      200000 non-null object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


In [17]:
df_tmp_test = prepare_data(df_test[:N])


In [18]:
df_tmp_test.head()

,text
0,Дизайнер-консультант мебели Обязанности: Рабо...
1,Продавец-консультант (ТЦ на Пушкина) Обязаннос...
2,Менеджер по продажам Торговый Дом «Форт» это в...
3,Продавец-консультант в магазин одежды (ТЦ Волн...
4,Специалист по охране труда Обязанности: осущ...


In [21]:
df_id = df_test[:N].drop(columns=['name', 'description'], axis=1)

In [22]:
df_id.head()

,id
0,200000
1,200001
2,200002
3,200003
4,200004


In [23]:
label = 'target'
RND_SEED = 123
idx_features = df_train.columns != label

In [24]:
X = df_train.loc[:, idx_features].values.tolist()
y = df_train.loc[:, ~idx_features].values
tests = df_tmp_test["text"].values

In [25]:
texts = []
for item in X:
    texts.append(item[0])

In [27]:
texts[1000]

'Технический специалист О компании: SUPPORT.UA - Мультикатегорийный сервис-провайдер № 1 в Украине. SUPPORT.UA - украинский дистрибьютор сервисов по установке, настройке и обслуживанию техники для розничных потребителей. Наша команда - это высококвалифицированные специалисты, которые предоставляют бесперебойный сервис высочайшего уровня. Support.ua ищет Технического Эксперта, специалиста по настройке компьютерной техники и мобильных устройств. Качества, которыми должен обладать наш сотрудник:  Желание работать в сфере настройки и обслуживания техники, развиваться и повышать свой уровень знаний на практике Навыки настройки цифровых устройств, знание Android, iOS, Windows Умение ориентироваться в различных категориях товара и способах оптимизации их работы Обучаемость, активность, коммуникабельность, нацеленность на результат  Мы предоставляем:  Официальное оформление с первого рабочего дня Интересную работу в динамично развивающейся компании Возможности для самореализации, профессиональ

In [28]:
model = Pipeline([
        ('union', FeatureUnion(
                # Use FeatureUnion to combine the features from subject and body
            transformer_list=[
                    # Pipeline for standard bag-of-words model
                ('text', Pipeline([
                        ('tfidf', TfidfVectorizer(
                            norm=None, 
                            smooth_idf=False,
                        )),
                    ])
                ),
            ],# end  transformer_list
                # weight components in FeatureUnion
            transformer_weights={
                'text': 0.5,
            }) # end FeatureUnion
        ),# end union
        ('clf', MultinomialNB()),
]) # end pipeline


In [29]:
%%time 
# Схема кросс-валидации
n_splits = 5
cv = StratifiedKFold(
        n_splits=n_splits, shuffle=True, 
        random_state=RND_SEED)

scores = cross_val_score(
    model, texts, y,
    scoring='roc_auc', 
    cv=cv, 
n_jobs=-1)

CPU times: user 5.7 s, sys: 11.1 s, total: 16.8 s
Wall time: 9min 19s


In [32]:
scores.mean(), scores.std()

(0.956702642505282, 0.0008412143581834525)

In [33]:
model.fit(texts, y)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_...former_weights={'text': 0.5})), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [34]:
predictions = model.predict(tests)

In [35]:
df_id['target'] = pd.DataFrame(predictions.tolist())

In [36]:
df_id.shape

(170179, 2)

In [42]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d_%H-%M')

In [43]:
df_id.to_csv(path_or_buf='./submission/submission_{today}.cvs'.format(today=today), sep=',')

'2018-03-09_09:16'